In [ ]:
!nvidia-smi || true

import torch
print("torch:", torch.__version__)
print("CUDA disponible:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

In [8]:
from IPython.display import display, Markdown

display(Markdown("""
# 1.1 Desplegar modelo abierto de Ollama
"""))


# 1.1 Desplegar modelo abierto de Ollama


In [3]:
!curl -sSfL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to render group...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> Enabling and starting ollama service...
Created symlink /etc/systemd/system/default.target.wants/ollama.service → /etc/systemd/system/ollama.service.
>>> NVIDIA GPU installed.


In [4]:
!ollama pull llama2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling 8934d96d3f08:   1% ▕                  ▏  37 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08:   2% ▕                  ▏  69 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08:   4% ▕                  ▏ 133 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08:   5% ▕                  ▏ 209 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08:   6% ▕█                 ▏ 245 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08:   8% ▕█                 ▏ 300 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08:   9% ▕█                 ▏ 351 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08:  10% ▕█                 ▏ 376 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08:  11% ▕██                ▏ 434 MB/3.8 GB                  pulling manifest 
pulling 8934d9

In [5]:
!pip install gradio requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 MB 148.9 MB/s  0:00:00m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 563.4/563.4 kB 39.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 139.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 152.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 226.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16/16 [gradio]15/16 [gradio]face-hub]]


In [6]:
import requests
import gradio as gr

OLLAMA_URL = "http://localhost:11434/api/generate"

def chat_fn(message, history):
    try:
        conversation = ""
        for h in history:
            if h["role"] == "user":
                conversation += f"Usuario: {h['content']}\n"
            elif h["role"] == "assistant":
                conversation += f"Asistente: {h['content']}\n"

        prompt = f"""
        Eres un asistente útil y amable. 
        Responde SIEMPRE en español, sin traducir al inglés.
        
        {conversation}
        Usuario: {message}
        Asistente:
        """

        payload = {"model": "llama2", "prompt": prompt, "stream": False}
        r = requests.post(OLLAMA_URL, json=payload)

        if r.status_code != 200:
            output = f"Error {r.status_code}: {r.text}"
        else:
            output = r.json().get("response", "")
    except Exception as e:
        output = f" Error: {str(e)}"

    history.append({"role": "user", "content": message})
    history.append({"role": "assistant", "content": output})
    return history, ""

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(type="messages", label="Chat en español")
    msg = gr.Textbox(placeholder="¿Que quieres preguntar?")
    state = gr.State([])

    msg.submit(chat_fn, [msg, state], [chatbot, msg])

demo.launch(server_name="0.0.0.0", server_port=7867, share=True)



* Running on local URL:  http://0.0.0.0:7867
* Running on public URL: https://c73cecce7e53a42d0d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [9]:
from IPython.display import display, Markdown

display(Markdown("""
# 1.2 Desplegar modelo 
"""))


# 1.2 Desplegar modelo 


In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

modelo = "HuggingFaceH4/zephyr-7b-beta"

tokenizador = AutoTokenizer.from_pretrained(modelo)
model = AutoModelForCausalLM.from_pretrained(
    modelo,
    device_map="auto",
    torch_dtype=torch.float16
)
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizador,
    max_new_tokens=512,
    temperature=0.7,
    top_p=0.9
)

def chat_fn(message, history):
    full_prompt = ""
    for h in history:
        full_prompt += f"[User]: {h[0]}\n[AI]: {h[1]}\n"
    full_prompt += f"[User]: {message}\n[AI]:"
    result = pipe(full_prompt, do_sample=True)
    reply = result[0]["generated_text"]
    if "[AI]:" in reply:
        reply = reply.split("[AI]:")[-1].strip()
    return reply

with gr.Blocks() as demo:
    display(Markdown("### Chat interactivo"))
    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="¿Que quieres preguntar?")
    state = gr.State([])

    def respond(message, history):
        reply = chat_fn(message, history)
        history.append((message, reply))
        return "", history

    msg.submit(respond, [msg, state], [msg, chatbot])
    demo.launch(server_name="0.0.0.0", server_port=7864, share=True)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the disk and cpu.
Device set to use cuda:0


### Chat interactivo

/tmp/ipykernel_19628/1195044786.py:34: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


* Running on local URL:  http://0.0.0.0:7864
* Running on public URL: https://74ed9a8450672c015b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
